In [7]:
import ccxt
import pandas as pd

# 初始化Binance客户端
binance = ccxt.binance()

# 获取BTC/USDT的历史数据
def fetch_data(symbol='ETH/USDT', timeframe='15m', limit=500):
    ohlcv = binance.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

data = fetch_data()
print(data.tail())

              timestamp     open     high      low    close     volume
495 2024-08-20 08:00:00  2674.81  2676.40  2665.11  2666.37  2575.8590
496 2024-08-20 08:15:00  2666.36  2667.74  2658.83  2661.34  1863.7497
497 2024-08-20 08:30:00  2661.35  2663.20  2657.40  2661.99  1702.2825
498 2024-08-20 08:45:00  2661.98  2664.87  2658.85  2661.20  1101.3328
499 2024-08-20 09:00:00  2661.20  2665.00  2660.41  2665.00  1156.9899


In [2]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=2, output_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model = LSTMModel()

In [11]:
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# 数据归一化
scaler = MinMaxScaler(feature_range=(-1, 1))
data['scaled_close'] = scaler.fit_transform(data['close'].values.reshape(-1, 1))

# 准备训练数据
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        label = data[i + seq_length]
        sequences.append((seq, label))
    return sequences

seq_length = 10
sequences = create_sequences(data['scaled_close'].values, seq_length)

# 转换为Tensor
# X = np.concatenate([s[0].unsqueeze(-1) for s in sequences], axis=0).astype(np.float32)
# y = np.concatenate([s[1].unsqueeze(-1) for s in sequences], axis=0).astype(np.float32)
# X = torch.from_numpy(X)
# y = torch.from_numpy(y)
X = torch.tensor([s[0] for s in sequences], dtype=torch.float32).unsqueeze(-1)
y = torch.tensor([s[1] for s in sequences], dtype=torch.float32).unsqueeze(-1)

train_dataset = TensorDataset(X, y)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 训练模型
epochs = 10
for epoch in range(epochs):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 0.0317
Epoch [2/10], Loss: 0.0139
Epoch [3/10], Loss: 0.0707
Epoch [4/10], Loss: 0.0060
Epoch [5/10], Loss: 0.0099
Epoch [6/10], Loss: 0.0093
Epoch [7/10], Loss: 0.0061
Epoch [8/10], Loss: 0.0068
Epoch [9/10], Loss: 0.0054
Epoch [10/10], Loss: 0.0041


In [12]:
import numpy as np

# 模拟持仓状态
current_position = None  # None, "long", "short"

# 模拟交易函数
def execute_trade(prediction, current_price, amount=0.001, symbol='BTC/USDT'):
    global current_position
    
    if prediction > current_price:
        if current_position == 'short':
            # 清空做空仓位
            print("清空做空仓位")
            binance.create_market_buy_order(symbol, amount)
            current_position = None
        if current_position is None:
            # 买入做多
            print(f"买入 {amount} BTC")
            binance.create_market_buy_order(symbol, amount)
            current_position = 'long'
    elif prediction < current_price:
        if current_position == 'long':
            # 清空做多仓位
            print("清空做多仓位")
            binance.create_market_sell_order(symbol, amount)
            current_position = None
        if current_position is None:
            # 卖出做空
            print(f"卖出 {amount} BTC")
            binance.create_market_sell_order(symbol, amount)
            current_position = 'short'

# 实时预测与交易
def trade():
    current_price = data['close'].iloc[-1]
    prediction = predict(model, data['scaled_close'].values, seq_length)
    predicted_price = scaler.inverse_transform([[prediction]])[0, 0]
    print(f"当前价格: {current_price}, 预测价格: {predicted_price}")
    
    execute_trade(predicted_price, current_price)

trade()

NameError: name 'predict' is not defined

In [ ]:
def update_model():
    new_data = fetch_data()  # 获取新的数据
    data = pd.concat([data, new_data]).drop_duplicates().reset_index(drop=True)
    
    # 更新训练集
    sequences = create_sequences(data['scaled_close'].values, seq_length)
    X = torch.tensor([s[0] for s in sequences], dtype=torch.float32).unsqueeze(-1)
    y = torch.tensor([s[1] for s in sequences], dtype=torch.float32).unsqueeze(-1)
    
    train_dataset = TensorDataset(X, y)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    # 再次训练模型
    for epoch in range(epochs):
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
        print(f'更新模型 - Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# 周期性更新模型
update_model()